In [1]:
from pyspark.sql import SparkSession

ImportError: No module named pyspark.sql

In [ ]:
spark = SparkSession.builder.appName('Project_new').getOrCreate()

In [ ]:
#read the data
df = spark.read.csv('Final.csv', inferSchema=True, header=True)

In [ ]:
#Schema of the dataset
df.printSchema()

In [ ]:
#For low recommended 311 severity
low_311 = df.filter("311_Severity_1 > (311_Severity_2 & 311_Severity_3)")

In [ ]:
#Medium recomended 
medium_311 = df.filter("311_Severity_2 > (311_Severity_3 & 311_Severity_1)")

In [ ]:
#Highly recommended
high_311 = df.filter("311_Severity_3 > (311_Severity_2 & 311_Severity_3)")

In [ ]:
#For low crime
low_crime = df.filter("Crime_Severity_1 > (Crime_Severity_2 & Crime_Severity_3)")

In [ ]:
#Medium crime condition
medium_crime = df.filter("Crime_Severity_2 > (Crime_Severity_3 & Crime_Severity_1)")

In [ ]:
#high crime condition
high_crime = df.filter("Crime_Severity_3 > (Crime_Severity_2 & Crime_Severity_1)")

In [ ]:
from pyspark.sql.functions import col, when, coalesce, lit

In [ ]:
#low is set as 1
df_low_311 = low_311.withColumn("311_Status", lit(1))

In [ ]:
#medium is set as 2
df_medium_311 = medium_311.withColumn("311_Status", lit(2))

In [ ]:
#high is set as 3
df_high_311 = high_311.withColumn("311_Status", lit(3))

In [ ]:
#Merged all the values to single column
df_new = df_low_311.union(df_medium_311)

In [ ]:
df_311 = df_new.union(df_high_311)

In [ ]:
#high crime is set as 1 which is low recommended
df_low_crime = low_crime.withColumn("Crime_Status", lit(1))

In [ ]:
#medium crime is set as 2 which is medium recommended
df_medium_crime = medium_crime.withColumn("Crime_Status", lit(2)

In [ ]:
#low crime is set as 3 which is highly recommended
df_high_crime = high_crime.withColumn("Crime_Status", lit(3))

In [ ]:
#Merged data values in single column
df_new_crime = df_low_crime.union(df_medium_crime)

In [ ]:
df_crime = df_new_crime.union(df_high_crime)

In [ ]:
df_311.head(1)

In [ ]:
df_crime.head(1)

In [ ]:
from pyspark.sql.functions import col

In [ ]:
#Performed inner join on data
final_df_1 = df_311.join(df_crime,df_311['311_Zip']== df_crime['311_Zip'], how = 'inner').alias("ZIPCODE")


In [ ]:
#Selected the columns that contained ZIPCODE,311_Status,Crime_Status,RES_RATING
final_df = df_311.join(df_crime,df_311['311_Zip']== df_crime['Crime_Zip'], how = 'cross').select(df_311['311_Zip'].alias("ZIPCODE"),df_311['311_Status'].alias("311_STATUS"),df_crime['Crime_Status'].alias("COMPLAINT_STATUS"),df_311['RATING'].alias("RES_RATING"))

In [ ]:
final_df.printSchema()

In [ ]:
low_zip = final_df.filter( (final_df['311_STATUS'] == 1) & (final_df['COMPLAINT_STATUS'] == 1)  &  (final_df['RES_RATING'] < 2.5) 
    |(final_df['311_STATUS'] == 1) & (final_df['COMPLAINT_STATUS'] == 2)  &  (final_df['RES_RATING'] < 2.5)
    |(final_df['311_STATUS'] == 2) & (final_df['COMPLAINT_STATUS'] == 1)  &  (final_df['RES_RATING'] < 2.5)
    |(final_df['311_STATUS'] == 2) & (final_df['COMPLAINT_STATUS'] == 2)  &  (final_df['RES_RATING'] < 2.5)
    |(final_df['311_STATUS'] == 3) & (final_df['COMPLAINT_STATUS'] == 1)  &  (final_df['RES_RATING'] < 2.5) 
    |(final_df['311_STATUS'] == 3) & (final_df['COMPLAINT_STATUS'] == 2)  &  (final_df['RES_RATING'] < 2.5)
    |(final_df['311_STATUS'] == 3) & (final_df['COMPLAINT_STATUS'] == 1)  &  (final_df['RES_RATING'] >= 2.5) & (final_df['RES_RATING'] < 3.3)
    |(final_df['311_STATUS'] == 2) & (final_df['COMPLAINT_STATUS'] == 1)  &  (final_df['RES_RATING'] >= 2.5) & (final_df['RES_RATING'] < 3.3)
    |(final_df['311_STATUS'] == 1) & (final_df['COMPLAINT_STATUS'] == 1)  &  (final_df['RES_RATING'] >= 2.5) & (final_df['RES_RATING'] < 3.3))

In [ ]:
medium_zip = final_df.filter( (final_df['311_STATUS'] == 2) & (final_df['COMPLAINT_STATUS'] == 2)  &  (final_df['RES_RATING'] >= 2.5) & (final_df['RES_RATING'] < 3.3)
    |(final_df['311_STATUS'] == 1) & (final_df['COMPLAINT_STATUS'] == 1)  &  (final_df['RES_RATING'] > 3.3)
    |(final_df['311_STATUS'] == 2) & (final_df['COMPLAINT_STATUS'] == 1)  &  (final_df['RES_RATING'] > 3.3)
    |(final_df['311_STATUS'] == 1) & (final_df['COMPLAINT_STATUS'] == 2)  &  (final_df['RES_RATING'] >= 2.5) & (final_df['RES_RATING'] < 3.3)
    |(final_df['311_STATUS'] == 3) & (final_df['COMPLAINT_STATUS'] == 2)  &  (final_df['RES_RATING'] >= 2.5) & (final_df['RES_RATING'] < 3.3)
    |(final_df['311_STATUS'] == 3) & (final_df['COMPLAINT_STATUS'] == 2)  &  (final_df['RES_RATING'] > 3.3)                      )

In [ ]:
high_zip  = final_df.filter( (final_df['311_STATUS'] == 3) & (final_df['COMPLAINT_STATUS'] == 3)  &  (final_df['RES_RATING'] >= 3.3) & (final_df['RES_RATING'] <= 5.0) 
                           |(final_df['311_STATUS'] == 3) & (final_df['COMPLAINT_STATUS'] == 1)  &  (final_df['RES_RATING'] >= 3.3) & (final_df['RES_RATING'] <= 5.0)
                           |(final_df['311_STATUS'] == 2) & (final_df['COMPLAINT_STATUS'] == 2)  &  (final_df['RES_RATING'] >= 3.3) & (final_df['RES_RATING'] <= 5.0)
                           |(final_df['311_STATUS'] == 1) & (final_df['COMPLAINT_STATUS'] == 2)  &  (final_df['RES_RATING'] >= 3.3) & (final_df['RES_RATING'] <= 5.0))

In [ ]:
low_zip = low_zip.withColumn("FINAL_STATUS", lit(0))
medium_zip = medium_zip.withColumn("FINAL_STATUS", lit(1))
high_zip = high_zip.withColumn("FINAL_STATUS", lit(2))


In [ ]:
output = low_zip.union(medium_zip)

In [ ]:
df_final_output = output.union(high_zip)

In [ ]:
df_final_output.printSchema()

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
assembler = VectorAssembler(inputCols=['ZIPCODE',
 '311_STATUS',
 'COMPLAINT_STATUS',
 'RES_RATING',
 ], outputCol='features')

In [ ]:
output = assembler.transform(df_final_output)

In [ ]:
data = output.select('features', 'FINAL_STATUS')

In [ ]:
train_data, test_data = data.randomSplit([0.7,0.3])

In [ ]:
#Applied logistic regression to train the model 
from pyspark.ml.classification import LogisticRegression

In [ ]:
status = LogisticRegression(labelCol='FINAL_STATUS')

In [ ]:
fitted_model = status.fit(train_data)

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
pred_and_labels = fitted_model.evaluate(test_data)

In [ ]:
pred_and_labels.predictions.show()

In [ ]:
#auc = model_eval.evaluate(pred_and_labels.predictions)
model_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='FINAL_STATUS')

In [ ]:
auc = model_eval.evaluate(pred_and_labels.predictions)

In [ ]:
auc

In [ ]:
#Applied decision trees to the model
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [ ]:
labelIndexer = StringIndexer(inputCol='FINAL_STATUS', outputCol='indexedLabel').fit(output)

In [ ]:
featuresIndexer = VectorIndexer(inputCol = 'features', outputCol='indexedFeatures', maxCategories=4).fit(output)

In [ ]:
(trainingData, testData) = output.randomSplit([0.7, 0.3])

In [ ]:
dt = DecisionTreeClassifier(labelCol='indexedLabel', featuresCol='indexedFeatures')

In [ ]:
pipeline = Pipeline(stages=[labelIndexer, featuresIndexer, dt])

In [ ]:
model = pipeline.fit(trainingData)

In [ ]:
print(model)

In [ ]:
predictions = model.transform(testData)

In [ ]:
testData.select('ZIPCODE').distinct().show(150)

In [ ]:
testData.filter("FINAL_STATUS == 2" ).select(['ZIPCODE']).distinct().show()

In [ ]:
predictions.select("prediction", "indexedLabel", "features").show(10)

In [ ]:
predictions.groupBy('prediction').count().show()

In [ ]:
labels='Best','average','low'
colors='red','green','blue'

In [ ]:
evaluator = MulticlassClassificationEvaluator(
labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
auc = evaluator.evaluate(predictions)

In [ ]:
treeModel = model.stages[2]

In [ ]:
print(treeModel)

In [ ]:
auc

In [ ]:
#from sklearn.datasets import load_iris

In [ ]:
#import sklearn.datasets as datasets
#import pandas as pd
#iris = datasets.load_iris()
#df = pd.DataFrame(iris.data, columns = iris.feature_names)
#y = iris.target

In [ ]:
#from sklearn.tree import DecisionTreeClassifier
#predictions = dt.DecisionTreeClassifier()
#predictions.fit(trainingData,testData)
#dtree = DecisionTreeClassifier()
#dtree.fit(df,y)

In [ ]:
#from sklearn.externals.six import StringIO

In [ ]:
#from IPython.display import Image

In [ ]:
#from sklearn.tree import export_graphviz

In [ ]:
#import pydotplus

In [ ]:
#dot_data = StringIO()

In [ ]:
#export_graphviz(dtree, out_file = dot_data,
 #              filled = True, rounded = True,
  #             special_characters = True)
#graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
#Image(graph.create_png())

 # Linear Regression

In [ ]:
assembler = VectorAssembler(inputCols=['ZIPCODE',
 '311_STATUS',
 'COMPLAINT_STATUS',
 'RES_RATING',
 ], outputCol='features')

In [ ]:
output = assembler.transform(df_final_output)

In [ ]:
data = output.select('features', 'FINAL_STATUS')

In [ ]:
data.show()

In [ ]:
train_data, test_data = data.randomSplit([0.7, 0.3])

In [ ]:
train_data.describe().show()

In [ ]:
test_data.describe().show()

In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
lr = LinearRegression(labelCol='FINAL_STATUS')

In [ ]:
trained_model = lr.fit(train_data)

In [ ]:
results = trained_model.evaluate(test_data)

In [ ]:
results.rootMeanSquaredError

In [ ]:
train_data.describe().show()

In [ ]:
results.r2

In [ ]:
results.meanSquaredError